In [1]:
import pandas as pd
import os # Good for managing file paths

# --- 1. Define your file paths ---
# Use 'r' before the string to make sure Python handles the backslashes correctly
path_oct = r"C:\Users\thist\Downloads\2019-Oct.csv.zip"
path_nov = r"C:\Users\thist\Downloads\2019-Nov.csv.zip"

# Define how many rows to read for our sample
sample_rows = 100000

print(f"Loading sample of {sample_rows} rows from October file...")

# --- 2. Load the October Data Sample ---
# We use compression='zip' because your file path ends in .zip
# We use nrows=sample_rows to get just the top part of the file
df_oct = pd.read_csv(path_oct, compression='zip', nrows=sample_rows)

print("--- October Data ---")
print(df_oct.info())
print("\n") # Adds a blank line
print(df_oct.head())
print("\n" + "="*50 + "\n") # Adds a separator


print(f"Loading sample of {sample_rows} rows from November file...")

# --- 3. Load the November Data Sample ---
df_nov = pd.read_csv(path_nov, compression='zip', nrows=sample_rows)

print("--- November Data ---")
print(df_nov.info())
print("\n")
print(df_nov.head())

Loading sample of 100000 rows from October file...
--- October Data ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     100000 non-null  object 
 1   event_type     100000 non-null  object 
 2   product_id     100000 non-null  int64  
 3   category_id    100000 non-null  int64  
 4   category_code  67413 non-null   object 
 5   brand          85607 non-null   object 
 6   price          100000 non-null  float64
 7   user_id        100000 non-null  int64  
 8   user_session   100000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 6.9+ MB
None


                event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00 UTC       view    44600062  2103807459595387724   
1  2019-10-01 00:00:00 UTC       view     3900821  2053013552326770905   
2  2019-10-01 00:00:01 UTC       

In [2]:
print("--- Analyzing October Sample (100,000 rows) ---")
print("\n")

# 1. See what event types exist and how many of each
print("Event Type Counts:")
event_counts = df_oct['event_type'].value_counts()
print(event_counts)
print("\n" + "="*50 + "\n")

# 2. See how many unique users are in this sample
unique_users = df_oct['user_id'].nunique()
print(f"Total unique users in this sample: {unique_users}")
print("\n" + "="*50 + "\n")

# 3. See how many unique sessions are in this sample
unique_sessions = df_oct['user_session'].nunique()
print(f"Total unique sessions in this sample: {unique_sessions}")

--- Analyzing October Sample (100,000 rows) ---


Event Type Counts:
event_type
view        97130
purchase     1655
cart         1215
Name: count, dtype: int64


Total unique users in this sample: 20384


Total unique sessions in this sample: 24382


In [3]:
print("--- Calculating Funnel for October Sample ---")

# 1. Get unique user IDs for each event type
# We use .unique() to get an array of users, then set() for fast operations
view_users = set(df_oct[df_oct['event_type'] == 'view']['user_id'].unique())
cart_users = set(df_oct[df_oct['event_type'] == 'cart']['user_id'].unique())
purchase_users = set(df_oct[df_oct['event_type'] == 'purchase']['user_id'].unique())

# 2. Calculate the funnel steps using set intersections
# This finds users who did ALL the steps in sequence.

# Step 1: Total users who viewed
total_viewers = len(view_users)

# Step 2: Users who viewed AND added to cart
# We find the intersection of the two sets
view_to_cart_users = view_users.intersection(cart_users)
total_cart_users = len(view_to_cart_users)

# Step 3: Users who viewed AND carted AND purchased
# We find the intersection of our previous group and the purchasers
all_steps_users = view_to_cart_users.intersection(purchase_users)
total_purchase_users = len(all_steps_users)

print(f"Total Viewers: {total_viewers}")
print(f"Viewers who also Carted: {total_cart_users}")
print(f"Viewers who Carted and Purchased: {total_purchase_users}")
print("\n" + "="*50 + "\n")

# 3. Calculate Conversion Rates
# We must check for division by zero, just in case
if total_viewers > 0:
    view_to_cart_rate = (total_cart_users / total_viewers) * 100
else:
    view_to_cart_rate = 0

if total_cart_users > 0:
    cart_to_purchase_rate = (total_purchase_users / total_cart_users) * 100
else:
    cart_to_purchase_rate = 0

print(f"View-to-Cart Conversion Rate: {view_to_cart_rate:.2f}%")
print(f"Cart-to-Purchase Conversion Rate: {cart_to_purchase_rate:.2f}%")
print("\n" + "="*50 + "\n")

# 4. Create the final DataFrame for Power BI
funnel_data = {
    'Step': ['1. View', '2. Add to Cart', '3. Purchase'],
    'User_Count': [total_viewers, total_cart_users, total_purchase_users]
}
funnel_df = pd.DataFrame(funnel_data)

print("Data for Power BI:")
print(funnel_df)

# 5. Save the data to a CSV file
funnel_df.to_csv('funnel_sample_for_powerbi.csv', index=False)
print("\nSuccessfully saved to 'funnel_sample_for_powerbi.csv'")

--- Calculating Funnel for October Sample ---
Total Viewers: 20383
Viewers who also Carted: 743
Viewers who Carted and Purchased: 431


View-to-Cart Conversion Rate: 3.65%
Cart-to-Purchase Conversion Rate: 58.01%


Data for Power BI:
             Step  User_Count
0         1. View       20383
1  2. Add to Cart         743
2     3. Purchase         431

Successfully saved to 'funnel_sample_for_powerbi.csv'


In [4]:
import pandas as pd

print("--- Loading FULL October dataset (this may take a minute)... ---")

# Define the file path
path_oct = r"C:\Users\thist\Downloads\2019-Oct.csv.zip"

# --- Smart Loading ---
# We only load the columns we need to save memory
columns_to_load = ['event_type', 'user_id']

df_full = pd.read_csv(path_oct, compression='zip', usecols=columns_to_load)

print("Full dataset loaded successfully.")
print(f"Total events processed: {len(df_full)}")
print("\n" + "="*50 + "\n")

print("--- Calculating Funnel for FULL October Data ---")

# 1. Get unique user IDs for each event type
# We'll use .loc[] for efficient filtering
view_users = set(df_full.loc[df_full['event_type'] == 'view', 'user_id'].unique())
cart_users = set(df_full.loc[df_full['event_type'] == 'cart', 'user_id'].unique())
purchase_users = set(df_full.loc[df_full['event_type'] == 'purchase', 'user_id'].unique())

# 2. Calculate the funnel steps
total_viewers = len(view_users)

view_to_cart_users = view_users.intersection(cart_users)
total_cart_users = len(view_to_cart_users)

all_steps_users = view_to_cart_users.intersection(purchase_users)
total_purchase_users = len(all_steps_users)

print(f"Total Viewers: {total_viewers}")
print(f"Viewers who also Carted: {total_cart_users}")
print(f"Viewers who Carted and Purchased: {total_purchase_users}")
print("\n" + "="*50 + "\n")

# 3. Calculate Conversion Rates
if total_viewers > 0:
    view_to_cart_rate = (total_cart_users / total_viewers) * 100
else:
    view_to_cart_rate = 0

if total_cart_users > 0:
    cart_to_purchase_rate = (total_purchase_users / total_cart_users) * 100
else:
    cart_to_purchase_rate = 0

print(f"View-to-Cart Conversion Rate: {view_to_cart_rate:.2f}%")
print(f"Cart-to-Purchase Conversion Rate: {cart_to_purchase_rate:.2f}%")
print("\n" + "="*50 + "\n")

# 4. Create the final DataFrame for Power BI
funnel_data_final = {
    'Step': ['1. View', '2. Add to Cart', '3. Purchase'],
    'User_Count': [total_viewers, total_cart_users, total_purchase_users]
}
funnel_df_final = pd.DataFrame(funnel_data_final)

print("Final Data for Power BI:")
print(funnel_df_final)

# 5. Save the final data
funnel_df_final.to_csv('funnel_FINAL_for_powerbi.csv', index=False)
print("\nSuccessfully saved to 'funnel_FINAL_for_powerbi.csv'")

--- Loading FULL October dataset (this may take a minute)... ---
Full dataset loaded successfully.
Total events processed: 42448764


--- Calculating Funnel for FULL October Data ---
Total Viewers: 3022130
Viewers who also Carted: 336996
Viewers who Carted and Purchased: 202754


View-to-Cart Conversion Rate: 11.15%
Cart-to-Purchase Conversion Rate: 60.17%


Final Data for Power BI:
             Step  User_Count
0         1. View     3022130
1  2. Add to Cart      336996
2     3. Purchase      202754

Successfully saved to 'funnel_FINAL_for_powerbi.csv'


In [5]:
import pandas as pd

print("--- Loading FULL October dataset (this may take a minute)... ---")

# Define the file path
path_oct = r"C:\Users\thist\Downloads\2019-Oct.csv.zip"

# --- Smart Loading ---
# We only load the columns we need to save memory
columns_to_load = ['event_type', 'user_id']

df_full = pd.read_csv(path_oct, compression='zip', usecols=columns_to_load)

print("Full dataset loaded successfully.")
print(f"Total events processed: {len(df_full)}")
print("\n" + "="*50 + "\n")

print("--- Calculating Funnel for FULL October Data ---")

# 1. Get unique user IDs for each event type
# We'll use .loc[] for efficient filtering
view_users = set(df_full.loc[df_full['event_type'] == 'view', 'user_id'].unique())
cart_users = set(df_full.loc[df_full['event_type'] == 'cart', 'user_id'].unique())
purchase_users = set(df_full.loc[df_full['event_type'] == 'purchase', 'user_id'].unique())

# 2. Calculate the funnel steps
total_viewers = len(view_users)

view_to_cart_users = view_users.intersection(cart_users)
total_cart_users = len(view_to_cart_users)

all_steps_users = view_to_cart_users.intersection(purchase_users)
total_purchase_users = len(all_steps_users)

print(f"Total Viewers: {total_viewers}")
print(f"Viewers who also Carted: {total_cart_users}")
print(f"Viewers who Carted and Purchased: {total_purchase_users}")
print("\n" + "="*50 + "\n")

# 3. Calculate Conversion Rates
if total_viewers > 0:
    view_to_cart_rate = (total_cart_users / total_viewers) * 100
else:
    view_to_cart_rate = 0

if total_cart_users > 0:
    cart_to_purchase_rate = (total_purchase_users / total_cart_users) * 100
else:
    cart_to_purchase_rate = 0

print(f"View-to-Cart Conversion Rate: {view_to_cart_rate:.2f}%")
print(f"Cart-to-Purchase Conversion Rate: {cart_to_purchase_rate:.2f}%")
print("\n" + "="*50 + "\n")

# 4. Create the final DataFrame for Power BI
funnel_data_final = {
    'Step': ['1. View', '2. Add to Cart', '3. Purchase'],
    'User_Count': [total_viewers, total_cart_users, total_purchase_users]
}
funnel_df_final = pd.DataFrame(funnel_data_final)

print("Final Data for Power BI:")
print(funnel_df_final)

# 5. Save the final data
funnel_df_final.to_csv('funnel_FINAL_for_powerbi.csv', index=False)
print("\nSuccessfully saved to 'funnel_FINAL_for_powerbi.csv'")

--- Loading FULL October dataset (this may take a minute)... ---
Full dataset loaded successfully.
Total events processed: 42448764


--- Calculating Funnel for FULL October Data ---
Total Viewers: 3022130
Viewers who also Carted: 336996
Viewers who Carted and Purchased: 202754


View-to-Cart Conversion Rate: 11.15%
Cart-to-Purchase Conversion Rate: 60.17%


Final Data for Power BI:
             Step  User_Count
0         1. View     3022130
1  2. Add to Cart      336996
2     3. Purchase      202754

Successfully saved to 'funnel_FINAL_for_powerbi.csv'


In [6]:
import os

# --- 1. Define your target folder and file name ---
save_directory = r"C:\Users\thist\Downloads"
file_name = "funnel_FINAL_for_powerbi.csv"

# --- 2. Create the full path (this is the best practice) ---
full_save_path = os.path.join(save_directory, file_name)

# --- 3. Save the DataFrame to that specific path ---
# (This assumes 'funnel_df_final' is still in your notebook's memory)
funnel_df_final.to_csv(full_save_path, index=False)

print(f"Successfully saved file to: {full_save_path}")

Successfully saved file to: C:\Users\thist\Downloads\funnel_FINAL_for_powerbi.csv
